# PageRank
Today we will implement PageRank algorithm for a small collection of document about Information Retrieval. For this we will extract link information from Wikipedia and build a Google Matrix. 

To obtain ranking result you can use:
- Naïve approach with matrix inversion
- Power Method

In [ ]:
!pip install wikipedia

In [ ]:
import wikipedia

pages = [
    "Bag-of-words model",
    "Bayes' theorem",
    "Cluster analysis",
    "Content-based image retrieval",
    "Database",
    "Deep learning",
    "Desktop search",
    "Dimensionality reduction",
    "Discounted Cumulative Gain",
    "Eigenvector",
    "Full-text search",
    "Gummy bear",
    "Hypertext",
    "Image retrieval",
    "Information system",
    "Internet",
    "K-nearest neighbors algorithm",
    "Language model",
    "Latent Dirichlet allocation",
    "Latent semantic analysis",
    "Low-rank approximation",
    "Multimedia information retrieval",
    "Netflix Prize",
    "Netflix",
    "Web query",
    "Ranking (information retrieval)",
    "Recommender systems",
    "Relevance (information retrieval)",
    "Rocchio algorithm",
    "Search algorithm",
    "Search engines",
    "Semantic search",
    "Semantic web",
    "Sentiment analysis",
    "Similarity search",
    "Site search",
    "Text mining",
    "Text Retrieval Conference",
    "Tf–idf",
    "Vector space model",
    "Web crawler",
    "World Wide Web"
]

import tqdm.notebook as tqdm

dataset = {}
for page in tqdm.tqdm(pages):
    if page not in dataset:
        dataset[page] = wikipedia.page(page)

## Essential data is stored in adjacency matrix

Here we create a 0/1 adjacency matrix.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

A = np.zeros((len(pages), len(pages)))
for j, page in enumerate(tqdm.tqdm(pages)):
    for link in dataset[page].links:
        if link in pages:
            i = pages.index(link)
            A[i, j] = 1

plt.imshow(A)
plt.show()

## Prepare a stochastic matrix M based on adjacency matrix A

In [ ]:
M = A.copy()
for j in range(len(pages)):
    if sum(A[:, j]) > 0:
        M[:, j] = A[:, j] / sum(A[:, j])
    else:
        M[:, j] = np.ones((len(pages),)) / len(pages)

In [ ]:
plt.imshow(M)
plt.show()
print(f"Values range from {M.min()} to {M.max()}")

## 1.2. Prepare Google matrix

$G_{ij}=\alpha \mathcal{M}_{ij}+(1-\alpha ){\frac {1}{N}}$

In [ ]:
def to_google(M, alpha=0.85):
    return M * alpha + np.ones(M.shape) * (1 - alpha) / M.shape[0]

In [ ]:
G = to_google(M)
print(f"Values range from {G.min():.4f} to {G.max():.4f}")

In [ ]:
plt.imshow(G)
plt.show()

## Solve naively

$\mathbf{R} =  (\mathbf{I}-d \mathcal{M})^{-1}  \frac{1-d}{N}  \mathbf{1}$

In [ ]:
I = np.eye(M.shape[0])
d = 0.85
N = M.shape[0]
_1 = np.ones((M.shape[0], 1))

R = np.matrix((I - d * M)).I @ _1 * ((1 - 0.85) / N)
ids = np.argsort(-R.reshape(-1)).A1
for idx in ids:
    print(f"{R[idx, 0]:.4f} {pages[idx]}")

## Solve with power method

$R = G^{N}v_{random}$

In [ ]:
v = np.random.rand(G.shape[0], 1)
v /= sum(v)

for i in range(35):
    v = G @ v

assert np.allclose(v, R), "Estimated vector should be close to naively computed"

ids = np.argsort(-v.reshape(-1))
for idx in ids:
    print(f"{v[idx, 0]:.4f} {pages[idx]}")

## 1.5. Build in check

In [ ]:
evals, evecs = np.linalg.eig(G)
evecs = abs(evecs)
ids = np.argsort(-evecs[:, 0])
for idx in ids:
    print(f"{evecs[idx, 0]:.4f} {pages[idx]}")